In [2]:
from transformers import AutoFeatureExtractor, ASTModel
import torch
import librosa
import joblib, pickle

feature_extractor = AutoFeatureExtractor.from_pretrained("bookbot/distil-ast-audioset")
model = ASTModel.from_pretrained("bookbot/distil-ast-audioset").to('cuda')

scaler = joblib.load('./robustscaler_AST.pkl')
pca = joblib.load('./pca_AST.pkl')
ocsvm = joblib.load('./ocsvm_AST.pkl')

In [4]:
data_path = './train/대화_57_43.wav'
y, sr = librosa.load(data_path, sr=16000)

import time
s_time = time.time()

input_tensor = feature_extractor(y, sampling_rate=sr, return_tensors="pt")
with torch.no_grad():
    feature = model(**input_tensor.to('cuda')).last_hidden_state.detach()
    feature = feature.cpu().numpy().reshape(1, -1)
    feature = scaler.transform(feature)
    feature = pca.transform(feature)
    pred = ocsvm.predict(feature)
    
print(time.time()-s_time)

0.10894584655761719


In [5]:
pred

array([1])

In [6]:
data_path = './test_data_1second/N-10_220817_B_2_a_25127_10.wav'
y, sr = librosa.load(data_path, sr=16000)

import time
s_time = time.time()

input_tensor = feature_extractor(y, sampling_rate=sr, return_tensors="pt")
with torch.no_grad():
    feature = model(**input_tensor.to('cuda')).last_hidden_state.detach()
    feature = feature.cpu().numpy().reshape(1, -1)
    feature = scaler.transform(feature)
    feature = pca.transform(feature)
    pred = ocsvm.predict(feature)
    
print(time.time()-s_time)

0.13402414321899414


In [7]:
pred

array([-1])